## Bayesian Regression

This notebook is for the Bayesian Regression model on the data!

In [1]:
# %load_ext autoreload # this ensures modules are reloaded automatically
# %autoreload 2 

In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from IPython.display import display
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Cleaning methods
import cleaning
import constants
import imputation
import auxiliary

# Scikit learn
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
from sklearn.impute import KNNImputer
from sklearn.linear_model import BayesianRidge

from scipy.spatial import Voronoi, voronoi_plot_2d
from matplotlib.patches import Rectangle, Circle
from matplotlib.colors import ListedColormap

from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

# For adaboost
from sklearn.ensemble import AdaBoostClassifier


import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/test.csv
data/example-submission.csv
data/train.csv
data/additional_data/consumer_price.csv
data/auxiliary-data/auxiliary-data/sg-primary-schools.csv
data/auxiliary-data/auxiliary-data/sg-secondary-schools.csv
data/auxiliary-data/auxiliary-data/sg-gov-markets-hawker-centres.csv
data/auxiliary-data/auxiliary-data/sg-train-stations.csv
data/auxiliary-data/auxiliary-data/sg-shopping-malls.csv
data/auxiliary-data/auxiliary-data/sg-commerical-centres.csv


In [3]:
# Main dataset
df_prices_train = pd.read_csv("data/train.csv")
df_prices_train_demo = pd.read_csv("data/train.csv")
df_prices_test = pd.read_csv("data/test.csv")

# Have a peek at the data
print("Property Prices train dataset", df_prices_train.shape)
display(df_prices_train.head())

Property Prices train dataset (26048, 23)


,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,...,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price
0,6998418,seascape,57 cove way,condominium,condominium,ocr,strata,3,4.0,4,...,1.239337,103.837487,leasehold/99 years,2011.0,151.0,2336.0,uncategorized,guarded,2021-05-04,5390000.0
1,2046604,la maison,10 moulmein rise,apartment,apartment,ocr,strata,3,3.0,11,...,1.319533,103.847030,freehold,1999.0,24.0,1259.0,uncategorized,guarded,2021-05-07,2310000.0
2,7563513,viva,2 suffolk walk,condominium,condominium,ocr,strata,4,3.0,11,...,1.315656,103.844445,freehold,2012.0,235.0,1959.0,uncategorized,guarded,2021-06-30,5279500.0
3,3071445,urban treasures,205 jalan eunos,condominium,condominium,ocr,strata,3,2.0,14,...,1.329367,103.905791,freehold,NaN,237.0,883.0,uncategorized,guarded,2022-01-02,1843600.0
4,9667539,infini at east coast,east coast road,apartment,apartment,ocr,strata,3,3.0,15,...,1.309176,103.911352,freehold,NaN,36.0,1066.0,uncategorized,guarded,2021-12-24,2262700.0


# Data Cleaning

In [4]:
# First, drop all fields in TO_IGNORE as these columns are either redundant
# e.g. we drop model because we have type
# or they only have one unique value, e.g. 'market_segment' and 'type_of_area'.
df_prices_train = df_prices_train.drop(columns=constants.TO_IGNORE)
df_prices_test = df_prices_test.drop(columns=constants.TO_IGNORE)

In [5]:
# Then, we clean up the tenure fields to 3 categories: 60 years, 99 years, and freehold/999
df_prices_train['tenure'] = df_prices_train['tenure'].apply(cleaning.tenure_to_binary)
df_prices_test['tenure'] = df_prices_test['tenure'].apply(cleaning.tenure_to_binary)
# Perform one hot encoding
df_prices_train = cleaning.categorical_to_onehot(df_prices_train, ['tenure', 'type', 'region', 'planning_area'])
df_prices_test = cleaning.categorical_to_onehot(df_prices_test, ['tenure', 'type', 'region', 'planning_area'])
# Remove 'planning_area_seletar' column in test as that's not seen in train.
df_prices_test.drop(columns=['planning_area_seletar'], inplace=True)

In [6]:
# Take 4+1 as 5
df_prices_train['bedrooms'] = df_prices_train['bedrooms'].apply(cleaning.process_bedroom_sum)
df_prices_test['bedrooms'] = df_prices_test['bedrooms'].apply(cleaning.process_bedroom_sum) 

# Take 4+1 as 4.5
# df_prices_train['bedrooms'] = df_prices_train['bedrooms'].apply(cleaning.process_bedroom_half)
# df_prices_test['bedrooms'] = df_prices_test['bedrooms'].apply(cleaning.process_bedroom_half) 
df_prices_train[df_prices_train['bedrooms'].isna()]

,bedrooms,bathrooms,district,lat,lng,no_of_units,area_size,price,tenure_60,tenure_99,...,planning_area_sembawang,planning_area_sengkang,planning_area_serangoon,planning_area_singapore river,planning_area_southern islands,planning_area_tampines,planning_area_tanglin,planning_area_toa payoh,planning_area_woodlands,planning_area_yishun
59,NaN,1.0,15,1.314620,103.932237,116.0,409.0,715000.0,0,0,...,0,0,0,0,0,0,0,0,0,0
111,NaN,NaN,6,1.292988,103.851047,39.0,6609.0,27423000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
203,NaN,1.0,2,1.274644,103.844742,360.0,365.0,990000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
296,NaN,1.0,2,1.274644,103.844742,360.0,365.0,1100000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
443,NaN,NaN,22,1.335557,103.742417,738.0,506.0,1045000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25849,NaN,1.0,18,1.376176,103.960488,473.0,571.0,693000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
25866,NaN,2.0,23,1.379727,103.760191,338.0,624.0,825000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
25884,NaN,1.0,1,1.277083,103.849181,510.0,441.0,1099800.0,0,1,...,0,0,0,0,0,0,0,0,0,0
25916,NaN,1.0,7,1.297510,103.856297,522.0,409.0,1098900.0,0,1,...,0,0,0,0,0,0,0,0,0,0


# Add Auxiliary Data (optional)

In [ ]:
# Import auxiliary data
df_commercial_centres = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-commerical-centres.csv")
df_train_stations = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-train-stations.csv")
df_shopping_malls = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-shopping-malls.csv")
df_primary_schools = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-primary-schools.csv")
df_secondary_schools = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-secondary-schools.csv")

# Have a peek at the data
print("Aux: Commercial Centres", df_commercial_centres.shape)
display(df_commercial_centres.head())
print("Aux: Train Stations", df_train_stations.shape)
display(df_train_stations.head())
print("Aux: Shopping Malls", df_shopping_malls.shape)
display(df_shopping_malls.head())
print("Aux: Primary Schools", df_primary_schools.shape)
display(df_primary_schools.head())
print("Aux: Secondary Schools", df_secondary_schools.shape)
display(df_secondary_schools.head())

We add some helper functions for computing the distances to these important locations and add the distances to our dataframes:

In [7]:
from geopy import distance
from tqdm import tqdm
tqdm.pandas()

# Helper Functions 
def distance_ll(lat1: float, lng1: float, lat2: float, lng2: float) -> float:
    return distance.distance((lat1, lng1), (lat2, lng2))

def dist_to_nearest_item(expr: pd.DataFrame, aux_df: pd.DataFrame) -> str:
    return round(min(aux_df.apply(lambda x: distance_ll(expr['lat'], expr['lng'], x['lat'], x['lng']), axis=1)).km, 1)

ModuleNotFoundError: No module named 'tqdm'

## Bayesian Regression

In [9]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = BayesianRidge()
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

(26048, 55)
n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 8682
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 5
n_resources: 8682
Fitting 10 folds for each of 5 candidates, totalling 50 fits
----------
iter: 1
n_candidates: 2
n_resources: 26046
Fitting 10 folds for each of 2 candidates, totalling 20 fits


In [10]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1912440.054176 using {'imputer__n_neighbors': 11}


In [11]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [12]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_bayesian_regression_no_aux_tenure_oridinal.csv")